# Obtaining Playlist Information

Using our Spotify client ID and secret, install and initialize Spotipy.

In [3]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
cid = '41ef3d43ab644b70b02c5cd59c863774'
secret = 'e6dc9a5208a04977991022b24a1bb6fe'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


We apply the below function to extract track data and store track ID.

In [4]:
def get_playlist_tracks(username,playlist_id):
    
    # Takes in a Spotify user name and playlist ID (URI) and returns list of dictionaries of track data
    
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    
    # Extending pagination results limit
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

all_tracks = get_playlist_tracks('Four Tet', '2uzbATYxs9V8YQi5lf89WG')

# Create list of track IDs
all_track_ids = []
for i in all_tracks:
    all_track_ids.append(i['track']['id'])
len(all_track_ids)

1749

Now that we have our IDs, we use Spotipy to extract additional data; in this case we will build a DataFrame with descriptive metadata and audio features.

In [5]:
def getTrackFeatures(id_list):
    
    # Create list of features to be used in conjuction with ID list
    meta = sp.track(id_list)
    features = sp.audio_features(id_list)

    # metadata
    track_name = meta['name']
    album = meta['album']['name']
    artist_name = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    popularity = meta['popularity']

    # audio features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    duration_ms = meta['duration_ms']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [track_name, album, artist_name, release_date, duration_ms, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence]
    return track

In [6]:
# Iterating through ID list and getting feature columns
# Note - This cell takes several minutes to run
tracks = []
for i in range(len(all_track_ids)):
    track = getTrackFeatures(all_track_ids[i])
    tracks.append(track)

# Creating the DataFrame and saving as csv file
df = pd.DataFrame(tracks, columns = ['track_name', 'album', 'artist_name', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'])
df.to_csv("fourtet.csv", sep = ',')

In [7]:
df

track_name  \
0     My Friend of Misery - From Jason's Riff Tapes   
1                              Magnet Pulls Through   
2                            Houston in Two Seconds   
3                            Lost in A Lonely World   
4                                 Conducted Rhythms   
...                                             ...   
1744                                  Es war einmal   
1745                                      Kinoshita   
1746                          Promises Of Fertility   
1747                                              3   
1748                         Earth's Magnetic Field   

                                                  album      artist_name  \
0                 Metallica (Remastered Deluxe Box Set)        Metallica   
1                                              Tortoise         Tortoise   
2                                    Music by Ry Cooder        Ry Cooder   
3                                The Best of the Ethics       The Ethics   
4                     Peanut Butter Wolf's Jukebox 45's  Various Artists   
...                                                 ...              ...   
1744                                          Sowiesoso          Cluster   
1745                                        Eph Reissue           Fridge   
1746  For Those Of You Who Have Never (And Also Thos...        Huerco S.   
1747                                            Get Out             Pita   
1748  Columbia-Princeton Electronic Music Center 196...  Various Artists   

     release_date  duration_ms  popularity  danceability  acousticness  \
0      2021-09-10       277892          42         0.750       0.87400   
1      1994-09-01       278040          22         0.541       0.13300   
2            1995       121826          32         0.256       0.91700   
3      2003-01-01       154866          40         0.504       0.92000   
4      2002-09-01       111000           0         0.827       0.32300   
...           ...          ...         ...           ...           ...   
1744         1976       325400           0         0.643       0.75800   
1745   2002-05-07       302693           1         0.623       0.13200   
1746   2016-06-10       415933           0         0.260       0.82100   
1747   2008-05-12       678080          13         0.228       0.00869   
1748   1998-01-01       848266           6         0.282       0.97400   

       energy  instrumentalness  liveness  loudness  speechiness    tempo  \
0     0.21000            0.9420    0.1110   -19.358       0.0368  109.650   
1     0.24500            0.7950    0.1100   -17.452       0.0339   68.398   
2     0.04580            0.0419    0.1250   -23.396       0.0602  171.818   
3     0.11600            0.0706    0.1030   -11.122       0.0351  105.175   
4     0.47200            0.6160    0.5600    -7.085       0.3300  105.942   
...       ...               ...       ...       ...          ...      ...   
1744  0.15600            0.9760    0.1040   -20.074       0.0419  117.372   
1745  0.94500            0.8950    0.3020    -5.775       0.0984  139.971   
1746  0.28400            0.8410    0.1100   -19.656       0.0367   61.829   
1747  0.61900            0.9280    0.1540    -7.414       0.0483  105.538   
1748  0.00385            0.0023    0.0798   -19.061       0.0575   60.470   

      valence  
0      0.4990  
1      0.0435  
2      0.1200  
3      0.1950  
4      0.8200  
...       ...  
1744   0.3140  
1745   0.6110  
1746   0.4250  
1747   0.2070  
1748   0.3490  

[1749 rows x 15 columns]